In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

hybrid_error = np.load('hybrid_error_MC.npy')
vanilla_error = np.load('vanilla_error_MC.npy')
#hybrid_loss = np.load('hybrid_loss.npy')
#vanilla_loss = np.load('vanilla_loss.npy')

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[12, 5.0])

ax[0].set_title('L2 error (using Monte Carlo Integration)')
ax[0].set_yscale('log')
ax[0].plot( np.arange(301), np.sqrt(hybrid_error))
ax[0].plot(np.arange(301), np.sqrt(vanilla_error))
ax[0].legend(['hybrid method', 'vanilla method'])

ax[0].vlines([50, 100, 150, 200, 250], 0, 1, colors='r', linestyles='dashed')

ax[0].vlines([50, 100, 150, 200, 250], 0, 1, colors='r', linestyles='dashed')



FileNotFoundError: [Errno 2] No such file or directory: 'hybrid_loss.npy'

In [16]:
### Plot Contourf of the solutions over time
### Make a movie

0.004502168856561184

87.75

32.9

58.849999999999994

55.775